<a href="https://colab.research.google.com/github/anshkedia-04/Cafe_FAQ_Chatbot/blob/main/Cafe_Chatbot_Working.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Install necessary libraries

In [5]:
!pip install -U langchain langchain-community langchainhub transformers accelerate huggingface_hub --quiet
!pip install -U sentence-transformers faiss-cpu --quiet
!pip install -U pypdf --quiet

### Upload data files

In [7]:
from google.colab import files

# Upload menu.pdf, faq.txt, offers.csv
uploaded = files.upload()

Saving faq.txt to faq (1).txt
Saving menu.pdf to menu (1).pdf
Saving offers.csv to offers (1).csv


### Load documents

In [9]:
from langchain_community.document_loaders import PyPDFLoader, TextLoader, CSVLoader

# Load all three file types
pdf_loader = PyPDFLoader("menu.pdf")
txt_loader = TextLoader("faq.txt")
csv_loader = CSVLoader("offers.csv")

# Load documents
docs = pdf_loader.load() + txt_loader.load() + csv_loader.load()
# print(f"Loaded {len(docs)} documents.")
print(docs)

[Document(metadata={'producer': 'Microsoft® Word 2021', 'creator': 'Microsoft® Word 2021', 'creationdate': '2025-05-14T23:53:41+05:30', 'author': 'ANSH KEDIA', 'moddate': '2025-05-14T23:53:41+05:30', 'source': 'menu.pdf', 'total_pages': 1, 'page': 0, 'page_label': '1'}, page_content='Brew Haven Café - Menu \n \n       Hot Beverages \n- Espresso ................... ₹120 \n- Cappuccino ................ ₹150 \n- Café Latte ................ ₹160 \n- Hot Chocolate ............. ₹180 \n- Masala Chai ............... ₹100 \n \n       Cold Beverages \n- Iced Americano ............ ₹130 \n- Cold Coffee ............... ₹150 \n- Iced Lemon Tea ............ ₹120 \n- Oreo Shake ................ ₹200 \n \n     Desserts \n- Chocolate Brownie ......... ₹110 \n- Cheesecake ................ ₹170 \n- Blueberry Muffin .......... ₹90 \n \n        Snacks \n- Veg Sandwich .............. ₹120 \n- Chicken Panini ............ ₹180 \n- Fries ..................... ₹90 \n \n   Vegan Options: Almond Milk, Soy Milk, 

### Split documents, create embeddings, and store in vector database

In [10]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

# Split into chunks
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
docs_split = splitter.split_documents(docs)

# Use sentence-transformers to create embeddings
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

# Store in vector DB
db = FAISS.from_documents(docs_split, embeddings)

### Load the Language Model

In [11]:
from langchain_community.llms import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline

# Load FLAN-T5 for Q&A
model_name = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

pipe = pipeline("text2text-generation", model=model, tokenizer=tokenizer, max_length=256)
llm = HuggingFacePipeline(pipeline=pipe)

Device set to use cuda:0


### Create Retriever and QA Chain

In [12]:
from langchain.chains import RetrievalQA

# Create retriever and QA chain
retriever = db.as_retriever(search_kwargs={"k": 3})
qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever)

### Ask Questions

In [13]:
# Ask questions in a loop
print("✅ Ask your café-related questions below. Type 'exit' to stop.\n")

while True:
    query = input("🔎 Your question: ")
    if query.lower() in ["exit", "quit", "stop"]:
        print("👋 Exiting. Thank you!")
        break
    try:
        response = qa_chain.invoke(query)
        print("💬 Answer:", response["result"], "\n")
    except Exception as e:
        print("❌ Error:", e, "\n")

✅ Ask your café-related questions below. Type 'exit' to stop.

🔎 Your question: Best sellers ?
💬 Answer: Cappuccino, Chicken Panini, and Cheesecake. 

🔎 Your question: What are the vegan options
💬 Answer: Almond Milk, Soy Milk, Vegan Sandwich 

🔎 Your question: What are the drinks available
💬 Answer: Hot Beverages - Espresso ................... 120 - Cappuccino ................ 150 - Café Latte ................ 160 - Hot Chocolate ............. 180 - Masala Chai ............... 100 Cold Beverages - Iced Americano ............ 130 - Cold Coffee ............... 150 - Iced Lemon Tea ............ 120 - Oreo Shake ................ 200 Desserts - Chocolate Brownie ......... 110 - Cheesecake ................ 170 

🔎 Your question: exit
👋 Exiting. Thank you!
